In [1]:
import gym
from gym.spaces import Discrete, Tuple, Box
import pybullet_envs
import pybullet

import ray
from ray import tune
from ray.tune import function
from ray.rllib.agents.ppo import PPOTrainer
from ray.tune.registry import register_env
import time

from low_level_env import LowLevelHumanoidEnv
from hier_env import HierarchicalHumanoidEnv

import timeit

from custom_callback import RewardLogCallback

import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
ray.shutdown()
ray.init(ignore_reinit_error=True)

2021-03-29 00:38:09,167	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.108',
 'raylet_ip_address': '192.168.0.108',
 'redis_address': '192.168.0.108:6379',
 'object_store_address': '/tmp/ray/session_2021-03-29_00-38-08_760603_88098/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-03-29_00-38-08_760603_88098/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-03-29_00-38-08_760603_88098',
 'metrics_export_port': 65163,
 'node_id': '4481f2c1e6edc8091b099d081921e63719cd436b3467c4f162debd50'}

In [3]:
def make_env_low(env_config):
    import pybullet_envs
    return LowLevelHumanoidEnv()

def make_env_hier(env_config):
    import pybullet_envs
    return HierarchicalHumanoidEnv()

def make_env(env_config):
    import pybullet_envs
    return gym.make('HumanoidBulletEnv-v0')



ENV = 'HumanoidBulletEnv-v0'
register_env(ENV, make_env)

ENV_LOW = 'HumanoidBulletEnv-v0-Low'
register_env(ENV_LOW, make_env_low)

ENV_HIER = "HumanoidBulletEnv-v0-Hier"
register_env(ENV_HIER, make_env_hier)

TARGET_REWARD = 2000

In [4]:
config = {
        "env": ENV,
        "num_workers": 0,
        "num_gpus": 1,
        "monitor": True,
        "evaluation_num_episodes": 50,
        "gamma": 0.995,
        "lambda": 0.95,
        "clip_param": 0.2,
        "kl_coeff": 1.0,
        "num_sgd_iter": 20,
        "lr": .0005,
        "sgd_minibatch_size": 8000,
        "train_batch_size": 24000,
        "model": {
            "fcnet_hiddens": [256, 128, 64],
            "fcnet_activation": "tanh",
            "free_log_std": True,
        },
        "batch_mode": "complete_episodes",
        "observation_filter": "MeanStdFilter",
    }

In [6]:
config_low = {
    "env": ENV_NAME,
    "num_workers": 0,
    "log_level": "WARN",
    "num_gpus": 1,
    "monitor": True,
    "evaluation_num_episodes": 50,
    "gamma": 0.995,
    "lambda": 0.95,
    "clip_param": 0.2,
    "kl_coeff": 1.0,
    "num_sgd_iter": 20,
    "lr": 0.0001,
    "sgd_minibatch_size": 8000,
    "train_batch_size": 24000,
    "model": {
        "fcnet_hiddens": [256, 128, 64],
        "fcnet_activation": "tanh",
        "free_log_std": True,
    },
    "batch_mode": "complete_episodes",
    "observation_filter": "MeanStdFilter",
}

NameError: name 'ENV_NAME' is not defined

In [7]:
def policy_mapping_fn(agent_id):
    if agent_id.startswith("low_level_"):
        return "low_level_policy"
    else:
        return "high_level_policy"

single_env_hier = HierarchicalHumanoidEnv()

highLevelPolicy = (
    None,
    single_env_hier.high_level_obs_space,
    single_env_hier.high_level_act_space,
    {
        "model": {
            "fcnet_hiddens": [64, 16],
            "fcnet_activation": "tanh",
            "free_log_std": True,
        },
    },
)

lowLevelPolicy = (
    None,
    single_env_hier.low_level_obs_space,
    single_env_hier.low_level_act_space,
    {
        "model": {
            "fcnet_hiddens": [256, 128, 64],
            "fcnet_activation": "tanh",
            "free_log_std": True,
        },
    },
)

config_high = {
    "env": ENV_HIER,
    "callbacks": RewardLogCallback,
    "num_workers": 0,
    "num_envs_per_worker": 1,
    "multiagent": {
        "policies": {
            "high_level_policy": highLevelPolicy,
            "low_level_policy": lowLevelPolicy,
        },
        "policy_mapping_fn": function(policy_mapping_fn),
    },
    "log_level": "WARN",
    "num_gpus": 1,
    "monitor": True,
    "evaluation_num_episodes": 50,
    "gamma": 0.995,
    "lambda": 0.95,
    "clip_param": 0.2,
    "kl_coeff": 1.0,
    "num_sgd_iter": 20,
    "lr": 0.001,
    "sgd_minibatch_size": 8000,
    "train_batch_size": 24000,
    "batch_mode": "complete_episodes",
    "observation_filter": "MeanStdFilter",
}

2021-03-29 00:38:42,478	WARNING sample.py:402 -- DeprecationWarning: wrapping <function policy_mapping_fn at 0x7f86f857d820> with tune.function() is no longer needed


In [8]:
agent = PPOTrainer(config_low)
# agent = PPOTrainer(config_high)
# agent = PPOTrainer(config)

2021-03-29 00:38:45,030	INFO trainer.py:616 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-03-29 00:38:45,030	INFO trainer.py:641 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-03-29 00:38:46,191	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'default_policy/log_std:0' shape=(17,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
2021-03-29 00:38:48,142	WARNING util.py:47 -- Install gputil for GPU system monitoring.


In [9]:
# Baseline
# agent.restore("/home/aditya/ray_results/HWalk/PPO_HumanoidBulletEnv-v0_881b4_00000_0_2021-02-21_17-45-29/checkpoint_710/checkpoint-710")

# Jalan dengan kaki kanan tegak
# agent.restore("/home/aditya/ray_results/HWalk_Low_Mimic/PPO_HumanoidBulletEnv-v0-Low_c0e9e_00000_0_2021-03-23_13-00-57/checkpoint_750/checkpoint-750")

# Frame skip 5
# agent.restore("/home/aditya/ray_results/HWalk_Low_Mimic/PPO_HumanoidBulletEnv-v0-Low_6a5d5_00000_0_2021-03-23_20-29-30/checkpoint_535/checkpoint-535")

# Frame skip 1
# agent.restore("/home/aditya/ray_results/HWalk_Low_Mimic/PPO_HumanoidBulletEnv-v0-Low_32439_00000_0_2021-03-23_22-15-18/checkpoint_545/checkpoint-545")


# Hierarcical
# agent.restore("/home/aditya/ray_results/HWalk_Hier_Mimic/PPO_HumanoidBulletEnv-v0-Hier_2aea3_00000_0_2021-03-25_15-10-23/checkpoint_1035/checkpoint-1035")

# Hierarchical fix frame inc
# agent.restore("/home/aditya/ray_results/HWalk_Hier_Mimic/PPO_HumanoidBulletEnv-v0-Hier_ece09_00000_0_2021-03-25_18-50-34/checkpoint_1010/checkpoint-1010")

# Hierarchical 2
# agent.restore("/home/aditya/ray_results/HWalk_Hier_Mimic/PPO_HumanoidBulletEnv-v0-Hier_c422a_00000_0_2021-03-25_22-31-20/checkpoint_2400/checkpoint-2400")

# Hierarchical (fix vector high level target)
# agent.restore("/home/aditya/ray_results/HWalk_Hier_Mimic/PPO_HumanoidBulletEnv-v0-Hier_95788_00000_0_2021-03-26_07-05-25/checkpoint_1220/checkpoint-1220")

# Low level agent
agent.restore("/home/aditya/ray_results/HWalk_Low_Mimic/PPO_HumanoidBulletEnv-v0-Low_e9c88_00000_0_2021-03-28_20-34-17/checkpoint_1100/checkpoint-1100")

2021-03-29 00:39:14,203	INFO trainable.py:371 -- Restored on 192.168.0.108 from checkpoint: /home/aditya/ray_results/HWalk_Low_Mimic/PPO_HumanoidBulletEnv-v0-Low_e9c88_00000_0_2021-03-28_20-34-17/checkpoint_1100/checkpoint-1100
2021-03-29 00:39:14,203	INFO trainable.py:379 -- Current state after restoring: {'_iteration': 1100, '_timesteps_total': None, '_time_total': 14489.67784833908, '_episodes_total': 580611}


In [10]:
agent.get_weights()['default_policy']['default_policy/fc_1/kernel'].shape
# agent.get_weights()['high_level_policy']['high_level_policy/fc_1/kernel'].shape

(42, 256)

In [72]:
env.close()

In [16]:
# env = make_env_hier(ENV_HIER)
env = make_env_low(ENV_LOW)
# env = make_env(ENV)

In [17]:
env.render()
observation = env.reset()

In [18]:
rew = []
for i in range(5):
    env.render()
    observation = env.reset()
    
    pybullet.removeAllUserDebugItems()
    pybullet.addUserDebugLine([0, 0, 0], [env.flat_env.walk_target_x, env.flat_env.walk_target_y, 0], lineColorRGB=[0, 1, 0], lineWidth=5)
    pybullet.addUserDebugLine([0, 0, 0], [1, 1, 0], lineColorRGB=[1, 1, 1], lineWidth=5)

    done = False
    tempRew = []
    startFrame = env.frame
    start = timeit.default_timer()
    while(not done):
        action = agent.compute_action(observation)
        observation, reward, done, info = env.step(action)
        # tempRew.append(env.calcJointScore())
        time.sleep(1.0/120)
    # stop = timeit.default_timer()
    # rew.append((startFrame, tempRew, stop-start))

# rew = []
# for i in range(5):
#     env.render()
#     observation = env.reset()
#     done = False
#     tempRew = []
#     startFrame = env.frame
#     start = timeit.default_timer()

#     pybullet.removeAllUserDebugItems()
#     pybullet.addUserDebugLine([0, 0, 0], [env.flat_env.walk_target_x, env.flat_env.walk_target_y, 0], lineColorRGB=[1, 0, 0], lineWidth=5)
    
    
#     while(not done):
#         action = dict()
#         if('high_level_agent' in observation):
#             action['high_level_agent'] = agent.compute_action(observation['high_level_agent'], policy_id='high_level_policy')
#         else:
#             action[env.low_level_agent_id] = agent.compute_action(observation[env.low_level_agent_id], policy_id='low_level_policy')
#         observation, reward, f_done, info = env.step(action)

#         basePos = env.flat_env.parts['torso'].get_position()
#         # pybullet.removeAllUserDebugItems()
#         # pybullet.addUserDebugLine([0, 0, 0], basePos, lineColorRGB=[0, 0, 1], lineWidth=5, lifeTime=0.2)
#         # pybullet.addUserDebugLine(basePos, basePos + env.targetHighLevel, lineColorRGB=[0, 1, 0], lineWidth=5, lifeTime=0.2)
#         done = f_done['__all__']
#         # tempRew.append(env.calcJointScore())
#         time.sleep(1.0/120)
#     # stop = timeit.default_timer()
#     # rew.append((startFrame, tempRew, stop-start))

In [19]:
env.close()

In [20]:
ray.shutdown()

In [25]:
a = np.zeros((3,))
b = np.ones((4,))
c = 3

In [26]:
np.hstack((c, a, b))

array([3., 0., 0., 0., 1., 1., 1., 1.])

In [53]:
a = +2

In [54]:
a

2